# Murder in Sequel City: Dig deeper and find the culprit!

![scrutinize](data/images/scrutinize.jpg)

## Goals

By the end of this lesson, you will be able to communicate with a SQL database using new language elements useful for grouping, filtering, and summarizing data. You will learn the following SQL elements:

* Common Table Expressions (CTE),
* Views,
* aggregate functions,
* `GROUP BY`,
* `HAVING`, and
* `CASE-WHEN`

## Introduction

**Business Context**: In the previous case, you decided to face the challenge of investigating a murder case. The evidence was saved in a relational database. It is time to unearth the clues stored in the database and dig deeper into the list of suspects. Sometimes things are not what they seem, and it is necessary to analyze the evidence objectively, but without losing the nose that, as an investigator, you have developed over the years. 

**Business Problem**: Based on the evidence stored in the database, you have to find out: **Who is the murderer?**

**Analytical Context**: In this case, we will examine data from a relational database consisting of 7 tables with an average of 10,000 records per table. This is the same database as in the first part of the case.

## The sign of the four

These are our four suspects. How about digging through that list to get more information?

|id|name|driver_id|address|ssn|income|id_1|age|gender|height|hair_color|eye_color|plate|car_make|car_model|car_model_year|
|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|
|45|Bron Fontanet|1008583170|70279 Ilene Hill|844-32-7352|59955.93|1008583170|78|Female|49|white|black|KSJ-33|Chevrolet|S10 Blazer|1992|
|146|Hercule Durram|3736669879|019 Sherman Center|500-48-6973|42626.13|3736669879|58|Female|66|red|brown|BFW-03|Chevrolet|Express|2009|
|647|Krista Fingleton|659712261|865 Mayfield Court|774-14-6683|71007.68|659712261|18|Male|48|white|brown|ONZ-43|Ford|Aerostar|1997|
|981|Tris MacVagh|9927362927|8706 Blue Bill Park Junction|232-23-7588|45976.29|9927362927|66|Male|50|white|green|ZDI-13|Maserati|Spyder|1989|

We've already learned about the [SQLite](https://www.sqlite.org/index.html) file format, which is a very popular format for small databases.

In order to load the SQLite file (which you can find at [`data/crime_database.db`](data/crime_database.db), we need to import the `sqlalchemy` Python library and run some code (below). Don't worry about learning that code; it's not SQL.

In [ ]:
%FETCH https://amzn-dana.workspace-lite.correlation-one.com/case.sql_basics_fellow/files/data/crime_database.db crime_database

In [ ]:
%LOAD crime_database RW

## The mysterious CTE affair

The query that returned the list of suspects was:

``` python
SELECT *
FROM individual
JOIN drivers ON individual.driver_id = drivers.id 
WHERE individual.id = 647 OR individual.id=146 OR individual.id=981 OR individual.id=45

```

The examples that follow will be based on this list of suspects. The point is that we need to temporarily store these results and be able to perform queries on them. To do this we will use a fairly new feature in the SQL language: [**Common Table Expressions (CTE)**](https://www.sqlite.org/lang_with.html), also known as "`WITH` queries". CTE allows us to (1) work with a set of rows for the duration of a single statement and (2) write subqueries making the overall SQL statement easier to read and understand.

The structure of a CTE is:

```sql
WITH cte_name AS (cte_definition)
SELECT …
FROM cte_name;

```
Basically, CTEs let us assign a temporary name to a query and then reference that name in another query to avoid having to write code that is too complex. The cell below shows how to define a CTE named `cte_suspects` with the query that extracts the suspects. Notice how once you define the CTE, it is possible to make another query on those results.

In [ ]:
WITH cte_suspects AS 
( 
    SELECT *
    FROM individual
    JOIN drivers ON individual.driver_id = drivers.id 
    WHERE individual.id = 647 OR individual.id=146 OR individual.id=981 OR individual.id=45
)
SELECT * FROM cte_suspects;

## The plaintiff's view

CTEs have a limited scope. They are restricted to the query where they are called right after being created. To permanently store the results of a query, you can use **views**, which are similar to tables, with the difference that you create them from a query (not by inserting new data into the database). Let's create a view to permanently store our list of suspects based on the query above.

In [ ]:
CREATE VIEW IF NOT EXISTS v_suspects AS
    SELECT *
    FROM individual JOIN drivers ON individual.driver_id = drivers.id 
    WHERE individual.id = 647 OR individual.id=146 OR individual.id=981 OR individual.id=45; 

Queries can be performed on views in the same way as with tables as shown below.

In [ ]:
SELECT *
FROM v_suspects;

## The `COUNT` of the Baskervilles

It could be useful for our investigation to explore some quick statistics of our suspect list, such as how many of them are female, or what is their average income. For this, SQL provides us with [**aggregate functions**](https://www.sqlitetutorial.net/sqlite-aggregate-functions/) such as `COUNT()`, `SUM()`, `AVG()`, `MIN()`, and `MAX()`.

### Example 1

How many of our suspects are female? With the knowledge we already have about SQL, we know that we just need to add a `WHERE` clause to filter the list of suspects by gender. 

In [ ]:
SELECT * FROM v_suspects
WHERE gender = 'Female';

There are two women in the data. This time it was easy-peasy because the number of rows was very small. But what if the table was very large? This is where aggregate functions come in handy. `COUNT` is one of such. You call it like this: `COUNT(*)`, and it returns the number of rows of a query. Execute the cell below to see it in action:

In [ ]:
SELECT COUNT(*) 
FROM v_suspects
WHERE gender = 'Female';

### Exercise 1
Write a query to find out how many drivers are recorded as male.

**Hint:** The relevant table is `drivers`.

**Answer.**

### Example 2

We often use aggregate functions like `COUNT` with a **`GROUP BY`** clause to group the results by one or more columns. A `GROUP BY` groups together rows that have the same values. It is always used after the `WHERE` clause.

Let's learn more about our suspects. What is the average income for men and women? To solve this question, we need to calculate the average using the `AVG` function and then group our suspects according to their gender using a `GROUP BY` clause, like below:

In [ ]:
SELECT gender, AVG(income)
FROM v_suspects
GROUP BY gender;

Let's change the name of the second column to make it a bit prettier. To assign a new column name in a query, we use the `AS` keyword:

In [ ]:
SELECT gender, AVG(income) AS average_income
FROM v_suspects
GROUP BY gender;

### Exercise 2

Use the`MAX` function to find the name of the oldest suspect. Use our list of suspects `v_suspects` and the `age` column.

**Answer.**

### Example 3
SQL allows us to add conditional logic to a query. For example, if we want to classify our suspects by age into three categories: young adult, adult, and senior, we can use the [`CASE - WHEN`](https://www.sqlitetutorial.net/sqlite-case/) pattern as shown in the following cell:

**Answer.** Shown below:

In [ ]:
SELECT name, 
       age,
       CASE
            WHEN (age >= 18 AND age <=25)
                THEN 'Young adult'
            WHEN (age > 25 AND age <65)
                THEN 'Adult'
            ELSE 'Senior' 
        END suspect_group
FROM 
    v_suspects;

To know how many suspects are in each category, we can add `COUNT` along with `GROUP BY` to the query. Below we wrote a `CASE-END` to create the `suspect_group` column and then counted how many records there are in each category with `GROUPBY` and `COUNT`:

In [ ]:
SELECT CASE
            WHEN (age >= 18 AND age <=25)
                THEN 'Young adult'
            WHEN (age > 25 AND age <65)
                THEN 'Adult'
            ELSE 'Senior' 
        END suspect_group,
        COUNT(*) as total
FROM 
    v_suspects
GROUP BY (suspect_group);

### Exercise 3

Write a query to classify all the people from the `individual` table according to their income brackets:

| income_group   | Boundaries                           |   |   |   |
|--------|----------------------------------|---|---|---|
| Low    | less than 35,000                  |   |   |   |
| Medium | between 35,000 and 50,000, inclusive |   |   |   |
| High   | More than 50,000                  |   |   |   |

How many people are there in each category?

**Answer.**

### Example 4

What if we wanted to filter the results of a grouped query? SQL allows us to add extra conditions to the grouped query with the `WHERE` clause.

Returning to Example 3, suppose we want to know how many *male* suspects there are by age range. In this case, we add the `WHERE` clause to the query before the `GROUP BY` like this:

In [ ]:
SELECT
       CASE
            WHEN (age >= 18 AND age <=25)
                THEN 'Young adult'
            WHEN (age > 25 AND age <65)
                THEN 'Adult'
            ELSE 'Senior' 
        END suspect_group,
        COUNT(*) as total
FROM 
    v_suspects
WHERE gender = 'Male'
GROUP BY (suspect_group);

---

SQL also allows us to add constraints on the *grouped* results with the **`HAVING`** clause. There is a difference between using the `HAVING` and `WHERE` clauses. Suppose we want to know, after adding up the incomes by gender, which gender has more than $105,000 collectively. Let's write the basic query using `SUM` and `GROUP BY`:

In [ ]:
SELECT SUM(income) as total_income, gender
FROM v_suspects
GROUP BY (gender);

Let's use SQL to determine which gender exceeds $105,000 (the answer is obvious in this very simple query, but that isn't always the case!).

We run the cell below and notice that if we filter the query with the `WHERE` clause, we get no results because the `WHERE` is filtering by the income of each row (individual values), and we know that *none* of our suspects earn more than $105,000. 

In [ ]:
SELECT SUM(income) as total_income, gender
FROM v_suspects
WHERE income > 105000
GROUP BY (gender);

If we use `HAVING` instead, we get the answer. Notice that we are using the aggregate column to make the filter, which means that *`HAVING` filters on the aggregate results*. In other words, `HAVING` applies the conditions to the grouped values, not the individual values in the individual rows (notice that the `HAVING` clause is always used after the `GROUP BY` clause).  

In [ ]:
SELECT SUM(income) as total_income, gender
FROM v_suspects
GROUP BY (gender)
HAVING total_income > 105000;

### Exercise 4
Write a query to calculate how many drivers there are by hair color. 

**Answer.**

### Exercise 5

Modify the above query to count how many drivers there are for each hair color category. Filter the results to only show those categories with more than 200 drivers.

**Answer.**

## [The final problem](https://en.wikipedia.org/wiki/The_Final_Problem)

Here we have all the data of our prime suspect: Tris MacVagh who has a car with a license plate beginning with ZDI. Let's go deeper into his testimony.

In [ ]:
SELECT *
FROM individual i JOIN drivers d ON i.driver_id = d.id 
WHERE i.id = 981;

In [ ]:
SELECT description
FROM interrogation
WHERE individual_id=981;

### Exercise 6

Follow the clues to discover the real killer. Start by writing a query that does a `JOIN` between the `individual` and `drivers` tables.

**Hint**. Use the `gender`, `hair_color`, `eye_color`, and `car_make` columns.

**Answer.**

### Exercise 7

Write a CTE query based on the query from the previous exercise. Name the query as `cte_prime_suspect`. Then write a `JOIN` with the `facebook_event` table based on the clues.

**Answer.**

Congratulations! You solved the mystery!

## Takeaways & conclusion

In this case we learned about:

**Common Table Expression (CTE)**
```sql
WITH cte_name AS (cte_definition)
SELECT …
FROM cte_name;

```
**Aggregate Functions**

- `COUNT()` - counts rows in a specified column or table
- `SUM()` - calculates the sum of values
- `AVG()` - calculates the average of a set of values
- `MIN()` - gets the minimum value in a set of values
- `MAX()` - gets the maximum value in a set of values

**`GROUP BY`** - Combines rows with identical values, often used in combination with aggregate functions

**`HAVING`** - Filters on aggregate results

**`CASE - WHEN`** -
```sql
CASE
    WHEN ComparsionCondition THEN result
    WHEN ComparsionCondition THEN result
    ELSE other
 END
```


## Attribution

"Scrutinize", Ethan Sees, CC-0, https://www.pexels.com/es-es/buscar/escudri%C3%B1ar/